In [1]:
import os
from scipy.interpolate import interp1d
import pandas as pd
import plotly.express as px
import numpy as np
import geopy

import airbnb_dataset
import config_data

In [ ]:
myABData = airbnb_dataset.AirbnbDataset()
ctry_cities = list(config_data.country_to_city.values())
state_cities = []
states = []
for k,v in config_data.US_state_to_city.items():
    state_cities.extend(v)
    for t in range(len(v)):
        states.append(k)

In [ ]:
#need to process reviews in a datewise manner
# one csv per month, across all locations
alldata=[]
cols = ["city","lat","long"]
for i in range(1,11):
    cols.append("month"+str(i))
print(cols)

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='myapplication')
def getlocation(city):
    location = geolocator.geocode(city)
    print(location)
    ns1 = float(location.raw['boundingbox'][0])
    ns2 = float(location.raw['boundingbox'][1])
    ew1 = float(location.raw['boundingbox'][2])
    ew2 = float(location.raw['boundingbox'][3])
    
    return (ns1+ns2)/2,(ew1+ew2)/2

In [ ]:
location = geolocator.geocode("Twin Cities Minneapolis")
print(location)

In [ ]:
for i,c in enumerate(state_cities):
    df = myABData.get_reviews_for_city(c)
    df19 = df.loc[2019].values[:-2]
    df20 = df.loc[2020].values
    rel_diff = [c]
    if c == 'Twin Cities MSA':
        c = c[:c.rindex(c)]
    
    lat,long = getlocation(c+' ' + states[i])
    rel_diff.append(lat)
    rel_diff.append(long)
    rel_diff.extend(list(((df19 - df20)/df19).T[0]))
    alldata.append(tuple(rel_diff))
    
monthlydf = pd.DataFrame(alldata, columns=cols)
monthlydf = monthlydf.set_index(["city"])
monthlydf.to_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/monthly_reviews.csv'))

In [ ]:
#weekly, 10 months
alldata = []
cols = ["city","lat","long"]

min_n_months = 1000
for i,c in enumerate(state_cities):
    df = myABData.get_reviews_for_city(c,per_week = True,per_month = False)
    df20 = df.loc[2020].values[:41]
    df19 = df.loc[2019].values[:41]
    rel_diff = [c]
    if c == 'Twin Cities MSA':
        c = c[:c.rindex(' ')]
    lat,long = getlocation(c + " "+states[i])
    rel_diff.append(lat)
    rel_diff.append(long)
    rel_diff.extend(list(((df19 - df20)/df19).T[0]))
    alldata.append(tuple(rel_diff))
for i in range(1,42):
    cols.append("week"+str(i))
weeklydf = pd.DataFrame(alldata,columns = cols)
weeklydf.to_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/weekly_reviews.csv'))

In [ ]:
#GLOBAL
cols = []
for i in range(1,42):
    cols.append("week"+str(i))
df = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'GLOBAL/weekly_reviews.csv'))

for col in cols:
    list1 = df[col].values.tolist()
    minval = min(list1)
    list1 = list(np.array(list1) - minval)
    m = interp1d([0,3.9],[1,50])
    circ_rad = m(list1)
    fig = px.density_mapbox(df, lat="lat", lon="long",center={'lat':17,'lon':25},radius=circ_rad,zoom=1,mapbox_style="stamen-watercolor",opacity=0.7,width=1000,height=700)
    #orange is most affected area
    fig.write_image(os.path.join(config_data.AIRBNB_DATA_DIR,f'GLOBAL/{col}.png'))

In [13]:
#US
cols = []
for i in range(1,42):
    cols.append("week"+str(i))
df = pd.read_csv(os.path.join(config_data.AIRBNB_DATA_DIR,'US/weekly_reviews.csv'))
for col in cols:
    list1 = df[col].values.tolist()
    minval = min(list1)
    list1 = list(np.array(list1) - minval)
    m = interp1d([0,1.8],[1,50])
    circ_rad = m(list1)
    fig = px.density_mapbox(df, lat="lat", lon="long",center={'lat':34,'lon':-110},radius=circ_rad,zoom=2.6,mapbox_style="stamen-watercolor",opacity=0.7,width=1000,height=700)
    #orange is most affected area
    fig.write_image(os.path.join(config_data.AIRBNB_DATA_DIR,f'US/{col}.png'))